## CNN MODEL KURULUMU

In [83]:
from pathlib import Path

import tensorflow as tf
from tensorflow.keras import layers, models

import scipy.io
import pandas as pd
from sklearn.model_selection import train_test_split


In [84]:


def collect_dataset_paths(root_dir):

    root = Path(root_dir)

    volume_paths = []   # .mha, .mah
    mat_paths    = []   # .mat
    img_paths    = []   # .jpg, .jpeg, .png

    # Tüm dosyaları dolaş
    for p in root.rglob("*"):
        if not p.is_file(): 
            continue
        suf = p.suffix.lower()
        if suf in (".mha", ".mah"):
            volume_paths.append(p)
        elif suf == ".mat":
            mat_paths.append(p)
        elif suf in (".jpg", ".jpeg", ".png"):
            img_paths.append(p)

    # Sıralama (opsiyonel, deterministik çıktı için)
    volume_paths = sorted(volume_paths)
    mat_paths    = sorted(mat_paths)
    img_paths    = sorted(img_paths)

    return volume_paths, mat_paths, img_paths

if __name__ == "__main__":
    root_folder = r"C:\Users\hdgn5\OneDrive\Masaüstü\Bilgisayar Proglamlama\images"
    volumes, mats, images = collect_dataset_paths(root_folder)

    print(f"► Toplam tıbbi hacim dosyası (.mha/.mah): {len(volumes)}")
    print(f"► Toplam .mat dosyası:                     {len(mats)}")
    print(f"► Toplam fotoğraf dosyası:                  {len(images)}\n")

    print("Örnek .mha/.mah dosyaları:")
    for p in volumes[:5]:
        print(" ", p.name)
    print("\nÖrnek .mat dosyaları:")
    for p in mats[:5]:
        print(" ", p.name)
    print("\nÖrnek fotoğraf dosyaları:")
    for p in images[:5]:
        print(" ", p.name)


► Toplam tıbbi hacim dosyası (.mha/.mah): 0
► Toplam .mat dosyası:                     6
► Toplam fotoğraf dosyası:                  14779

Örnek .mha/.mah dosyaları:

Örnek .mat dosyaları:
  Abyssinian_100.mat
  Abyssinian_101.mat
  Abyssinian_102.mat
  Abyssinian_100.mat
  Abyssinian_101.mat

Örnek fotoğraf dosyaları:
  Abyssinian_1.jpg
  Abyssinian_10.jpg
  Abyssinian_100.jpg
  Abyssinian_101.jpg
  Abyssinian_102.jpg


In [85]:
import scipy.io
from pathlib import Path


def inspect_one_mat(mat_path):
    data = scipy.io.loadmat(mat_path)
    print(f"Inspecting {mat_path.name}:")
    for k in data.keys():
        # MATLAB’in gizli meta anahtarlarını atlayalım
        if not k.startswith("__"):
            print(" ", k, "→", type(data[k]), data[k].shape if hasattr(data[k], "shape") else "")
    return data

if __name__ == "__main__":
    root = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Bilgisayar Proglamlama\images")
    mats = sorted(root.rglob("*.mat"))
    if mats:
        inspect_one_mat(mats[0])
    else:
        print("Hiç .mat dosyası bulunamadı.")


Inspecting Abyssinian_100.mat:
  frames → <class 'numpy.ndarray'> (4, 175288)
  binsa → <class 'numpy.ndarray'> (1, 175288)


In [86]:
import pandas as pd
from pathlib import Path

def build_image_label_df(img_paths):
    """
    Img dosya adlarından (örneğin 'Abyssinian_123.jpg') 
    sınıf etiketini ('Abyssinian') çıkartır ve bir DataFrame oluşturur.
    """
    records = []
    for p in img_paths:
        # stem: 'Abyssinian_123', split('_')[0] → 'Abyssinian'
        cls = p.stem.split('_')[0]
        records.append({
            'path': str(p),
            'label': cls
        })
    return pd.DataFrame.from_records(records)

if __name__ == "__main__":
    root_folder = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Bilgisayar Proglamlama\images")
    # Önceki adımda elde ettiğiniz img_paths listesini buraya alın:
    _, _, img_paths = collect_dataset_paths(root_folder)
    
    df = build_image_label_df(img_paths)
    print(df['label'].value_counts())     # Sınıf başına kaç resim var?
    print(df.head())


label
american         800
english          800
Abyssinian       400
basset           400
beagle           400
Birman           400
Bengal           400
British          400
chihuahua        400
Egyptian         400
Bombay           400
japanese         400
german           400
great            400
havanese         400
leonberger       400
keeshond         400
Maine            400
miniature        400
samoyed          400
newfoundland     400
Persian          400
pomeranian       400
pug              400
Ragdoll          400
Russian          400
saint            400
yorkshire        400
shiba            400
Siamese          400
Sphynx           400
wheaten          400
boxer            399
scottish         398
staffordshire    382
Name: count, dtype: int64
                                                path       label
0  C:\Users\hdgn5\OneDrive\Masaüstü\Bilgisayar Pr...  Abyssinian
1  C:\Users\hdgn5\OneDrive\Masaüstü\Bilgisayar Pr...  Abyssinian
2  C:\Users\hdgn5\OneDrive\Masaüstü\Bi

In [87]:
num_classes = df['label'].nunique()
print(f"Toplam sınıf sayısı: {num_classes}")


Toplam sınıf sayısı: 35


In [88]:
# 1. Cat vs Dog mapping
cat_breeds = {
    "Abyssinian", "Birman", "Bengal", "Bombay", "British",
    "Egyptian", "Maine", "Persian",  # yaygın kedi ırkları
    "american", "english"            # dataset’teki 'american', 'english' etiketleri kedi altgrubuna denk geliyor
}

# (Geriye kalan tüm etiketleri köpek kabul edeceğiz.)

# 2. DataFrame’inize binary label ekleme
import pandas as pd

# Örneğin df, daha önce oluşturduğunuz şöyle bir DataFrame:
#    path                                     label
# 0  .../Abyssinian_1.jpg                     Abyssinian
# 1  .../chihuahua_23.jpg                     chihuahua
# ...

df["binary_label"] = df["label"].apply(lambda x: 0 if x in cat_breeds else 1)
# 0 → kedi, 1 → köpek

print(df["binary_label"].value_counts())
# Çıktı örneği:
#  1    10000   # köpek sayısı
#  0     4779   # kedi sayısı


binary_label
1    9979
0    4800
Name: count, dtype: int64


In [89]:
def collect_image_paths(root_dir):
    """
    root_dir içindeki tüm .jpg/.jpeg/.png dosyalarını toplayıp döner.
    """
    root = Path(root_dir)
    exts = (".jpg", ".jpeg", ".png")
    img_paths = [p for p in root.rglob("*") if p.suffix.lower() in exts]
    return sorted(img_paths)


In [90]:
def build_df(img_paths):
    """
    Dosya adından (ör. 'Abyssinian_123.jpg') sınıf etiketini
    ('Abyssinian') çıkarır.
    """
    records = []
    for p in img_paths:
        cls = p.stem.split('_')[0]    # dosya adı: 'Breed_###'
        records.append({"path": str(p), "label": cls})
    df = pd.DataFrame.from_records(records)
    return df

In [91]:
def add_binary_label(df):
    """
    Kedi ırklarının kümesini kullanarak 0=kedi, 1=köpek kodunu ekler.
    """
    cat_breeds = {
        "Abyssinian", "Birman", "Bengal", "Bombay", "British",
        "Egyptian", "Maine", "Persian",
        "american", "english"
    }
    df["binary_label"] = df["label"].apply(lambda x: 0 if x in cat_breeds else 1)
    return df


In [92]:
def split_df(df, test_size=0.2, random_state=42):
    return train_test_split(
        df,
        test_size=test_size,
        stratify=df["binary_label"],
        random_state=random_state
    )


In [93]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0

    # Burada cast
    label = tf.cast(label, tf.int32)
    return img, label


In [94]:
def df_to_dataset(df, batch_size=BATCH_SIZE):
    paths  = df["path"].values
    labels = df["binary_label"].values      # => int64
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.shuffle(len(df))
    ds = ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
    return ds.batch(batch_size).prefetch(AUTOTUNE)


In [95]:
def build_binary_cnn_functional(
    input_shape=(*IMG_SIZE, 3),
    conv_filters=(32, 64, 128),
    kernel_size=3,
    pool_size=2,
    dropout_rate=0.5,
    fc_units=(256, 128, 64, 32),
):
    inputs = layers.Input(shape=input_shape)
    x = inputs
    for f in conv_filters:
        x = layers.Conv2D(f, kernel_size, padding="same", activation="relu")(x)
        x = layers.MaxPooling2D(pool_size)(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(dropout_rate)(x)
    for units in fc_units:
        x = layers.Dense(units, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return Model(inputs, outputs, name="binary_cnn_functional")

In [96]:
# --- 5. Ana akış ---
if __name__ == "__main__":
    root_dir = r"C:\Users\hdgn5\OneDrive\Masaüstü\Bilgisayar Proglamlama\images"
    img_paths = collect_image_paths(root_dir)
    df = build_df(img_paths)
    df = add_binary_label(df)
    train_df, test_df = split_df(df)

    train_ds = df_to_dataset(train_df)
    test_ds  = df_to_dataset(test_df)

    # Modeli inşa et ve eğit
    model = build_binary_cnn_functional()
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    model.summary()

    history = model.fit(
        train_ds,
        validation_data=test_ds,
        epochs=10
    )

    loss, acc = model.evaluate(test_ds)
    print(f"\nTest Loss: {loss:.4f}, Test Accuracy: {acc:.4f}")

    # Eğitimin sonunda kaydet
    model.save("binary_cnn_functional_256_128_64_32.h5")
    print("Model kaydedildi: binary_cnn_functional_256_128_64_32.h5")

Model: "binary_cnn_functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,525,377 (32.52 MB)

 Trainable params: 8,525,377 (32.52 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
370/370 ━━━━━━━━━━━━━━━━━━━━ 194s 491ms/step - accuracy: 0.6625 - loss: 0.6347 - val_accuracy: 0.7006 - val_loss: 0.5884
Epoch 2/10
370/370 ━━━━━━━━━━━━━━━━━━━━ 176s 475ms/step - accuracy: 0.6866 - loss: 0.5936 - val_accuracy: 0.6955 - val_loss: 0.5793
Epoch 3/10
370/370 ━━━━━━━━━━━━━━━━━━━━ 183s 495ms/step - accuracy: 0.7181 - loss: 0.5584 - val_accuracy: 0.7304 - val_loss: 0.5486
Epoch 4/10
370/370 ━━━━━━━━━━━━━━━━━━━━ 202s 547ms/step - accuracy: 0.7678 - loss: 0.4870 - val_accuracy: 0.7700 - val_loss: 0.4899
Epoch 5/10
370/370 ━━━━━━━━━━━━━━━━━━━━ 192s 517ms/step - accuracy: 0.8418 - loss: 0.3702 - val_accuracy: 0.8271 - val_loss: 0.4202
Epoch 6/10
370/370 ━━━━━━━━━━━━━━━━━━━━ 186s 501ms/step - accuracy: 0.9012 - loss: 0.2486 - val_accuracy: 0.8826 - val_loss: 0.3200
Epoch 7/10
370/370 ━━━━━━━━━━━━━━━━━━━━ 194s 523ms/step - accuracy: 0.9464 - loss: 0.1517 - val_accuracy: 0.9110 - val_loss: 0.2947
Epoch 8/10
370/370 ━━━━━━━━━━━━━━━━━━━━ 208s 561ms/step - accuracy: 0.9651 -


Test Loss: 0.3792, Test Accuracy: 0.9350
Model kaydedildi: binary_cnn_functional_256_128_64_32.h5
